# Bellman-equation

## Motivation

we have learned how reinforcement learning is about leaning a good (or optimal) path of actions in situations of sequential decision making

The "simplest" way of solving this problem is sampling all actions

As there are more actions complexity explodes

Remainder of the class is about how to deal with such complexity
- Efficiently sampling (learning from previous data) (e.g. off-policy learning/ causality)
- Efficiently finding "optimal" policies (e.g. Bellman Equations)
- Efficiently summarizing what we have "learned" (e.g. deep reinforcement learning) and generalizing

### Efficiency of the Bellman Equation - dynamic programming

Lets denote the number of states with $n_s$ and the number of actions with $n_a$.
The dynamic programming method is guaranteed to find the optimal policy in **polynomial time** (of $n_s$ and $n_a$).
The total number of possible policies are $n_a^{n_s}$. Therefore the dynamic programming approach is **exponentially faster than any direct search e.g. Monte Carlo Tree search** in policy space.


In practice, the dynamic programming approach can solve MDP problems (to be defined in what follows) with **millions of states**. This is important because if you manage to formalize  a problem in a tabular setting than it can be solved by dynamic programming and you can take advantage of its good convergence properties.

## State-value, action-value

**V - state-value function**

**Q - action-value function**


### State-value function (shortly: value-function)

The $\pi$ policy generates trajectories ($\tau$) until the end of the episode, starting from $s_0$.
$\tau = [s_0, a_0, r_0, s_1, a_1, r_1, ..., s_i, a_i, r_i, ..., s_T, a_T, r_T]$

$$V^\pi(s) = E_\tau \left[ G(\tau) | s_0 = s, \pi \right]$$

Where $G$ is the return. If the discounted return is used:

$$V^\pi(s) = E_\tau \left[ \sum_i{\gamma^i r_i} | s_0 = s, \pi, r_i \in \tau \right].$$

### Action-value function

The $\pi$ policy generates trajectories ($\tau$) until the end of the episode, starting from $s_0$.
$\tau = [s_0, a_0, r_0, s_1, a_1, r_1, ..., s_i, a_i, r_i, ..., s_T, a_T, r_T]$

$$Q^\pi(s, a) = E_\tau \left[ G(\tau) | s_0 = s, a_0 = a, \pi \right]$$

Where $G$ is the return. If the discounted return is used ($\gamma < 1$):

$$Q^\pi(s, a) = E_\tau \left[ \sum_i{\gamma^i r_i} | s_0 = s, a_0 = a, \pi, r_i \in \tau \right].$$

## Monte Carlo solution

The most naiv approach would be sampling trajectories and calculate the returns for each of them. Then it is possible to average them out and find the value function for each state:

$$V(s) = \frac{\sum_i^n{G(\tau_i)}}{n}$$

However, we can use a trajectory for sampling return for all the states encountered during the trajectory.
There are two strategies how to do that: first visit and every visit.

First visit means, if a state encountered several times during a trajectory, the return is calculated for only the first visit.

Every visit means, if a state encountered several times, the returns are calculated for each of them.


<img src="http://drive.google.com/uc?export=view&id=1uboWLi-NoQ1GMUZtrF1DBsc4Rxqx1BTU" width=65%>

The $s_i$ and $s_j$ is the same state but encountered after $i$ and $j$ steps.
The return gathered after state $s_i$:

$$G_i = \sum_{k=i}^T{ r(s_k) }$$

$$G_j = \sum_{k=j}^T{ r(s_k) }$$

For the first-visit MC, only $G_i$ is considered while for every-visit MC both $G_i$ and $G_j$ is considered.

## Dynamic programming solution

The Bellman-equation was named after Richard E. Bellman who applied this approach for engineering control problems.

*Almost any problem that can be solved using optimal control theory can also be solved by analyzing the appropriate Bellman equation. However, the term 'Bellman equation' usually refers to the dynamic programming equation associated with discrete-time optimization problems. In continuous-time optimization problems, the analogous equation is a partial differential equation that is usually called the Hamilton–Jacobi–Bellman equation.* (See [Wikipedia](https://en.wikipedia.org/wiki/Bellman_equation))

### Bellman-equation

Derivation of the Bellman-equation (mathematical version):

$$V^\pi(s) = E_\tau \left[ G(\tau) | s_0 = s, \pi \right]$$
$$V^\pi(s) = E_\tau \left[ r_1 + \gamma r_2 + \gamma^2 r_3 + ... | s_0 = s, \pi \right]$$
$$V^\pi(s) = E_\tau \left[ r_1 + \gamma (r_2 + \gamma r_3 + ...) | s_0 = s, \pi \right]$$
$$V^\pi(s) = E_\tau \left[ r_1 + \gamma G(\tau / s_0) | s_0 = s, \pi \right]$$
The expectation is an affine operation:
$$V^\pi(s) = E_\tau \left[ r_1 \right] + \gamma E_\tau \left[ G(\tau / s_0) | s_0 = s, \pi \right]$$
The expected value of a scalar is the scalar itself:
$$V^\pi(s) = E_\tau \left[ r_1 + \gamma E_\tau \left[ E_\tau G(\tau / s_0) | s_0 = s, \pi \right] \right]$$
$$V^\pi(s) = E_\tau \left[ r_1 + \gamma V^\pi(s_1) | s_0 = s, \pi \right]$$

[link text](https://)Understanding the Bellman-equation:

<img src="http://drive.google.com/uc?export=view&id=1J-jj-LQLnKIMFy9KV6cdKONtXQlMHhWI" width=70%>

Understanding the Bellman-equation:

The Bellman-equation is the result of the dynamic programming description of the trajectory. In a nutshell, we can think of dynamic programming as recursion: in order to get the current value, we can rely on the previous results. If we have a well defined initial value, base, then we can calculate the actual value. E.g.:

$H(k+1) = f(H(k))$

and $H(0)$ is known or trivial. $f$ is a function which is the connection between the current and the previous result. $H$ is the function of interest we would like to calculate.

Of course, this is not a general description of dynamic programming, just a short example.

How does this look like for the RL setting?

Relying on the previous example, we can write:

$$V^\pi(s) = f(V^\pi(N(s)))$$

where $N(s)$ is the set of states which is the neighborhood of the state $s$. Neighborhood means, the states available directly (with higher probability than 0) from state $s$.

What is the $f$ function?

For the trajectory of an agent, the Markov-property is true as we discussed earlier. Therefore it is not required to know the whole sequence to calculate the current state's value.

It is enough to know the current state and the transition probabilities to make one step forward.

Instead of using the recursion (backward direction), we can use a forward view. We can ask, if I know the value of all the neighboring states (possible next states), then what is the value of the current state.

<img src="http://drive.google.com/uc?export=view&id=16RAhhRENpOgFv_F6MgVdECrP63Bw0HRx" width=70%>

Formalizing the Bellman-equation (intuitive version):

* Current state: $s$
* A next state: $s' \in N(s)$
* Next action: $a$
* Immediate reward when moving from s with a: $r(s, a)$

For a single transition, $s \rightarrow a \rightarrow r \rightarrow s'$:

$$v(s) = r(s, a) + \gamma V(s')$$

$v(s)$ is for showing this is for only one path. Then we need the expected value of the $v(s)$ values.
Therefore we want to know the probability of the transition, shown above:

$$p(s, a, s') = \pi(s, a) \cdot T(s, a, s')$$

Formalizing the Bellman-equation (intuitive version):

Then the expected value:

$$V^\pi(s) = \sum_{a \in A}{\sum_{s'\in N(s)}{ p(s, a, s') \cdot v(s)}}$$

After putting them together:
$$V^\pi(s) = \sum_{a \in A}{\sum_{s'\in N(s)}{ \pi(s, a) T(s, a, s') \cdot \left[ r(s, a) + \gamma V^\pi(s') \right]}}$$

The previous formulation is true for:
* state-value function
* stochastic policy (known, or fixed)

As a consequence, there are several forms of the Bellman-equation. Let's go through them!

**What is the Bellman-equation for the state-value function with a deterministic policy?**

State-value, deterministic, fixed policy:

$$V^\pi(s) = \sum_{s'\in N(s)}{T(s, \pi(s), s') \cdot \left[ r(s, \pi(s)) + \gamma V^\pi(s') \right]}$$

**What is the Bellman-equation for the action-value function with a stochastic policy?**

$$Q^\pi(s, a) = \sum_{s'}{T(s, a, s') \cdot \left[ r(s, a) + \gamma \cdot V^\pi(s') \right]}$$


### Examples

Calculate the $V$ and $Q$ values for states, marked with question mark. The rewards are recieved when the agent enters to the state. There are two actions, according to which state the agent wants to go. The transition probabilities are the shown beside the arrows and it is the probability of the transition if the action triggers move to that state. The policy is the random policy. $\gamma = 0.8$.

**Example 1**

<img src="http://drive.google.com/uc?export=view&id=1kyp0TwU3OsCoRWI7gnd56AlFhkDFqkM9" width=65%>

**Example 2**

<img src="http://drive.google.com/uc?export=view&id=1hRJHlGQgCZy42170feEWALHyWBcJuC0G" width=65%>

**Example 3**

<img src="http://drive.google.com/uc?export=view&id=1sMXIbM7969JquYXrIB7kmMrX0U6RhqUI" width=65%>

**What is the connection between $Q$ and $V$?**

Try to answer this!

**What is the connection between $Q$ and $V$?**

$$V^\pi(s) = \sum_a{\pi(s, a) \cdot Q^\pi(s, a)}$$

and

$$Q^\pi(s, a) = \sum_{s'}{ T(s, a, s') \cdot \left[ r(s, a) + \gamma V^\pi(s') \right] }$$

**Calculating the policy**

If the state-value function is known (**requires the transition matrix**):

$$\pi'(s) = \arg\max_a\left( \sum_{s'}T(s, a, s') \cdot \left[ r(s, a) + \gamma V^\pi(s') \right]\right)$$

If the action-value function is known:

$$\pi'(s) = \arg\max_a\left( Q^\pi(s, a) \right)$$

As you can see, only deterministic policies can be calculated from the value-functions. $\pi' = \pi$ if the Q-function (and V) is the optimal one.

### Solution analysis

So far we have discussed a dynamic programming relation among states for the state-value and action-value functions.
The next three question we would like to address is, how to find:

* the solution (V or Q) for a given policy
* how to improve the policy
* how to find the optimal policy

We will answer this questions later on. Regarding the first point, the natural question is: Is the solution unique?

Banach's fix-point theorem (brief):

<img src="http://drive.google.com/uc?export=view&id=1drPT32Y6HKG1r8k11LFMOYUbX0AWWimw" width=65%>

Assume we have a vector space ($Z$) and we have a defined distance metric $d$ (e.g. euclidian-distance). The operation $F$ is a contraction if the following holds:

$$d(F(s), F(t)) < d(s, t)\ \forall s, t \in Z$$

For such an $F$ contraction operator, there exists **exactly one point** (fix point):

$$x = F(x)$$

**Proof by contradiction**

If we would have two fix points:

$$x_1 = F(x_1)$$
and
$$x_2 = F(x_2)$$

Then:

$$d(F(x_1), F(x_2)) < d(x_1, x_2)$$

Because the inequality is strict, this is a contradiction, $x_1$ and $x_2$ can not be different.

How does this connect to the Bellman-equation?

Take the V(s) function as a vector (discrete states and finite). Then one can reformalize the Bellman-equation:

$$\overline{V} = \overline{R} + \gamma P \overline{V} = F^\pi_\gamma\left( \overline{V} \right)$$

It can be proven, that $F^\pi_\gamma$, $\gamma < 1$ is a contraction (L2-norm). Therefore there is only one, unique $V$ for solution. The same is true for $Q$. $P$ contains the $\pi$ and $T$.



Which is the Bellman equation written in a vectorized form

$$
\left[\begin{array}{c}
v_\pi(1) \\
\vdots \\
v_\pi(n)
\end{array}\right]=\left[\begin{array}{c}
\mathcal{R}_1^\pi \\
\vdots \\
\mathcal{R}_n^\pi
\end{array}\right]+\gamma\left[\begin{array}{ccc}
\mathcal{P}_{11}^\pi & \ldots & \mathcal{P}_{1 n}^\pi \\
\vdots & \ddots & \vdots \\
\mathcal{P}_{n 1}^\pi & \ldots & \mathcal{P}_{n n}^\pi
\end{array}\right]\left[\begin{array}{c}
v_\pi(1) \\
\vdots \\
v_\pi(n)
\end{array}\right]
$$

Based on the below form of the value function

$$
v_\pi(s)=\sum_{a \in \mathcal{A}} \pi(a \mid s)\left(\mathcal{R}_s^a+\gamma \sum_{s^{\prime} \in \mathcal{S}} \mathcal{P}_{s s^{\prime}}^a v_\pi\left(s^{\prime}\right)\right)
$$

where

$$
\mathcal{R}_s^\pi=\sum_{a \in \mathcal{A}} \pi(a \mid s) \mathcal{R}_s^a
$$

and

$$
\mathcal{P}_{s s^{\prime}}^\pi=\sum_{a \in \mathcal{A}} \pi(a \mid s) \mathcal{P}_{s s^{\prime}}^a
$$



### Matrix solution

The Bellman-equation in a matrix form:

$$\overline{V} = \overline{R} + \gamma P \overline{V}$$

Then, we can solve this directly:

$$\overline{V} = \left( I - \gamma P \right)^{-1}\overline{R}$$

This is a really appealing solution. Seems easy. But it has $O(n^3)$ computational complexity if the state space contains $n$ states. Therefore it is only useful, if:

* the state space is small enough
* the reward function is known
* the transition probability (the dynamics of the environemnt) is known.

Furthermore this method, still not answeres how to find the optimal policy. This is just the calculation of the V (or Q) functions.

### Iterative solution

The solution has the following phases:
1. Initialize the policy $\pi$ and value table $V$
2. Calculate $V^\pi$ for the current $\pi$
3. Improve the policy according to $V^\pi$
4. Repeat until convergence

This can be used to find the solution. Below we will focus on step 2 and 3. For step 2, the matrix solution can be used as well. This method is similar for $Q$.

**Policy evaluation**

Due to the fact that the Bellman-equation can be considered as a contraction operation, the repeated use of the operator will get the value of $V$ closer to the solution (the fix point), step-by-step.

Therefore, the following equation should be evaluated several times, until the $V$ does not change significantly:

$$V^\pi_{t+1}(s) = \sum_{s'\in N(s)}{ T(s, \pi(s), s') \cdot \left[ r(s, \pi(s)) + \gamma V^\pi_t(s') \right]}$$

**Policy improvement (policy improvement theorem)**

After knowing the $V$ or $Q$, calculate the best policies suggested by $V$:

$$\pi'(s) = \arg\max_a\left( \sum_{s'\in N(s)}{ T(s, a, s') \cdot \left[ r(s, a) + \gamma V^\pi(s') \right]}\right)$$

Then:

$$\pi \leftarrow \pi'$$

**Policy improvement**

The key question is whether the policy $\pi'$ is better than $\pi$. The **policy improvement theorem** answers this:

*Let $\pi$ and $\pi'$ be any pair of deterministic policies such that, for all $s \in S$*,

$$Q^\pi(s, \pi'(s)) \geq V^\pi(s)$$

*Then the policy $\pi'$ must be as good as, or better than, $\pi$:*

$$V^{\pi'}(s) \geq V^\pi(s)$$

**Proof of policy improvement**

Keep expanding the $Q^\pi(s, \pi'(s))$:

$$V^\pi(s) \leq Q^\pi(s, \pi'(s))$$
$$V^\pi(s) \leq E_\pi\left[ r(s, a) + \gamma V^\pi(s_1) | s = s_0, a = \pi'(s) \right]$$
$$V^\pi(s) \leq E_{\pi'}\left[ r(s, a) + \gamma V^\pi(s_1) | s = s_0 \right]$$
$$V^\pi(s) \leq E_{\pi'}\left[ r(s, a) + \gamma Q^\pi(s_1, \pi'(s_1)) | s = s_0 \right]$$
$$V^\pi(s) \leq E_{\pi'}\left[ r(s, a) + \gamma E_{\pi'}\left[ r(s_1, a_1) + \gamma V^\pi(s_2) \right] | s = s_0 \right]$$
$$V^\pi(s) \leq E_{\pi'}\left[ r(s, a) + \gamma r(s_1, a_1) + \gamma^2 V^\pi(s_2) | s = s_0 \right]$$
$$V^\pi(s) \leq E_{\pi'}\left[ r(s, a) + \gamma r(s_1, a_1) + \gamma^2 r(s_2, a_2)  + \gamma^3 V^\pi(s_2) | s = s_0 \right]$$
$$V^\pi(s) \leq E_{\pi'}\left[ r(s, a) + \gamma r(s_1, a_1) + \gamma^2 r(s_2, a_2)  + \gamma^3 r(s_3, a_3) + ... | s = s_0 \right]$$
$$V^\pi(s) \leq V^{\pi'}(s)$$

**Policy iteration**

<img src="http://drive.google.com/uc?export=view&id=1CupNVaOMh5H3ikPIV9rllXkKdoasBzPb" width=75%>

**Value iteration**

It is possible to fuse the policy evaluation and policy improvement and apply only one operation at a time:

$$V^*_{t+1}(s) = \max_a{\sum_{s'\in N(s)}{T(s, a, s') \cdot \left[ r(s, a) + \gamma V^*_t(s') \right]}}$$

**Value iteration**

<img src="http://drive.google.com/uc?export=view&id=1NdOv59iJULMNDv8yt7itaNLSV8OIVWfG" width=75%>

### Complexity of dynamic programming (revisited)

- Recall that the Bellman equation is a particular type of dynamic programming

**Time Complexity**
In Dynamic programming problems, Time Complexity is the number of unique states/subproblems * time taken per state.

In this problem, for a given n, there are n unique states/subproblems. For convenience, each state is said to be solved in a constant time. Hence the time complexity is O(n * 1).

**Space Complexity**
We use one array called cache to store the results of n states. Hence the size of the array is n. Therefore the space complexity is O(n).


Note that there are trade-offs between complexity in space and time


For large problems (many states and many actions) the Bellman formulation is still not efficient.
However (as shown above), it is more efficient than the Monte Carlo method.


[source](https://www.freecodecamp.org/news/demystifying-dynamic-programming-24fbdb831d3a/#:~:text=In%20Dynamic%20programming%20problems%2C%20Time,O(n%20*%201))

## Bellman-equation summary/recap

<img src="http://drive.google.com/uc?export=view&id=1J-jj-LQLnKIMFy9KV6cdKONtXQlMHhWI" width=75%>

Value-function, fixed stochastic policy
$$V^\pi(s) = \sum_{s', a}{\pi(s, a) \cdot T(s, a, s') \cdot \left[ r(s, a) + \gamma \cdot V^\pi(s') \right]}$$

Value-function, fixed deterministic policy
$$V^\pi(s) = \sum_{s'}{T(s, \pi(s), s') \cdot \left[ r(s, \pi(s)) + \gamma \cdot V^\pi(s') \right]}$$

Action value-function, fixed stochastic policy
$$Q^\pi(s, a) = \sum_{s'}{T(s, a, s') \cdot \left[ r(s, a) + \gamma \cdot V^\pi(s') \right]}$$


Optimal value-function
$$\tilde{V}(s) = \max_a \sum_{s'}{T(s, a, s') \cdot \left[ r(s, a, s') + \gamma \cdot \tilde{V}(s') \right]}$$

Optimal action value-function
$$\tilde{Q}(s, a) = \sum_{s'}{T(s, a, s') \cdot \left[ r(s, a, s') + \gamma \cdot \max_{a'} \tilde{Q}(s', a') \right]}$$